In [ ]:
from __future__ import print_function, division
import os
import re
import copy
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec

import scipy
# from scipy import interpolate, sparse

import icesat2_toolkit.fit
import icesat2_toolkit.time
import icesat2_toolkit.utilities

from classify_photons import classify_photons
# from yapc.classify_photons import classify_photons


import h5py
import warnings
import copy

In [ ]:
# dset = pd.DataFrame()

# # print(fi['gt1l']['heights'].keys())
# # <KeysViewHDF5 ['delta_time', 'dist_ph_across', 'dist_ph_along', 'h_ph', 'lat_ph', 'lon_ph', 'pce_mframe_cnt', 'ph_id_channel', 'ph_id_count', 'ph_id_pulse', 'quality_ph', 'signal_conf_ph']>

# with h5py.File(fileID, 'r') as fi:
#     for key in fi['gt1l']['heights'].keys():
#         dset.insert('key', key[:])

In [ ]:
fileID06 = "228878251/ATL06_20200212232313_07370602_005_01.h5"
mds06 = {}
with h5py.File(fileID06, 'r') as fi:

    print(fi['gt1l'].keys())
    for top_key in fi[beam].keys():
        if (top_key != 'signal_find_output'): ### if instance of group?
            mds[top_key] = {}

                #ph_lat, ph_lon, etc.
            for key in fi[beam][top_key].keys():
                dataset = fi[beam][top_key].get(key) #(val) dataset will get closed w file
                mds[top_key][key] = dataset[:] #so copying dataset into memory
    
    # for key in fi['gt1l']['heights'].keys():
    #     dset.insert('key', key[:])

In [ ]:
def readATL03(fileID, beam, bbox):
    
    mds = {}
    
    with h5py.File(fileID, 'r') as fi:

        for top_key in fi[beam].keys():
            if (top_key != 'signal_find_output'): ### if instance of group?
                mds[top_key] = {}

                #ph_lat, ph_lon, etc.
                for key in fi[beam][top_key].keys():
                    dataset = fi[beam][top_key].get(key) #(val) dataset will get closed w file
                    mds[top_key][key] = dataset[:] #so copying dataset into memory
                    
        # print(min(mds['heights']['lon_ph']),max(mds['heights']['lon_ph']))  
        # print(min(mds['heights']['lat_ph']),max(mds['heights']['lat_ph']))        

        ##SUBSETTING HEIGHT AND GEOLOCATION:
        
        #if a lat/lon bounding box:
        if (len(bbox) == 4):
            w,e,s,n = bbox
        
            #boolean mask of all heights values (across keys) that satisfy bbox restrictions
            mask = (mds['heights']['lon_ph'] >= w) & (mds['heights']['lon_ph'] <= e) & \
                    (mds['heights']['lat_ph'] >= s) & (mds['heights']['lat_ph'] <= n)
        
        #else if a dt bounding box:        
        elif (len(bbox) == 2):
            dt_min, dt_max = bbox
            
            mask = (mds['heights']['delta_time'] >= dt_min) & (mds['heights']['delta_time'] <= dt_max) 
        
        #for pond 1 e.g. goes from 10730648 'heights' values to 23033 with the mask :)
        for k, v in mds['heights'].items():
            mds['heights'][k] = v[mask]    
            
        
        dt_min, dt_max = min(mds['heights']['delta_time']), max(mds['heights']['delta_time'])

        geo_mask = (mds['geolocation']['delta_time'] >= dt_min) & (mds['geolocation']['delta_time'] <= dt_max) 
        for k, v in mds['geolocation'].items():
            mds['geolocation'][k] = v[geo_mask]
        
        atlas_mask = (mds['bckgrd_atlas']['delta_time'] >= dt_min) & (mds['bckgrd_atlas']['delta_time'] <= dt_max) 
        for k, v in mds['bckgrd_atlas'].items():
            mds['bckgrd_atlas'][k] = v[atlas_mask]
        
        mds['orbit_info'] = {}
        for key,val in fi['orbit_info'].items(): 
            mds['orbit_info'][key] = val[:]
            
        mds['ancillary_data'] = {}
        ancillary_keys = ['atlas_sdp_gps_epoch','data_end_utc','data_start_utc',
            'end_cycle','end_geoseg','end_gpssow','end_gpsweek','end_orbit',
            'end_region','end_rgt','granule_end_utc','granule_start_utc','release',
            'start_cycle','start_geoseg','start_gpssow','start_gpsweek',
            'start_orbit','start_region','start_rgt','version']
        for key in ancillary_keys:
            #-- get each HDF5 variable
            mds['ancillary_data'][key] = fi['ancillary_data'][key][:]
                
         #-- transmit-echo-path (tep) parameters
        mds['ancillary_data']['tep'] = {}
        for key,val in fi['ancillary_data']['tep'].items():
            #-- get each HDF5 variable
            mds['ancillary_data']['tep'][key] = val[:]
       
        #-- channel dead time and first photon bias derived from ATLAS calibration
        cal1,cal2 = ('ancillary_data','calibrations')
        for var in ['dead_time','first_photon_bias']:
            mds[cal1][var] = {}
            for key,val in fi[cal1][cal2][var].items():
                #-- get each HDF5 variable
                if isinstance(val, h5py.Dataset):
                    mds[cal1][var][key] = val[:]
                elif isinstance(val, h5py.Group):
                    mds[cal1][var][key] = {}
                    for k,v in val.items():
                        mds[cal1][var][key][k] = v[:]

                #-- get ATLAS impulse response variables for the transmitter echo path (TEP)
                tep1,tep2 = ('atlas_impulse_response','tep_histogram')
                mds[tep1] = {}
                for pce in ['pce1_spot1','pce2_spot3']:
                    mds[tep1][pce] = {tep2:{}}
                    #-- for each TEP variable
                    for key,val in fi[tep1][pce][tep2].items():
                        mds[tep1][pce][tep2][key] = val[:]
        
        #-- Return the datasets and variables before HDF5 file closes automatically (not sure if indent needed)
        return mds

# Read ATL03 file for a bounding box

In [ ]:
w,s,e,n= -108.42635020791396, 38.788639821001155, -107.68485163209928, 39.34603060272382
bbox = w,e,s,n
beam = 'gt1r'

fileID = "225517609/ATL03_20200212232313_07370602_005_01.h5"
gt1_mds = readATL03(fileID, beam, bbox)

In [ ]:
plt.plot(gt1_mds['heights']['lon_ph'], gt1_mds['heights']['lat_ph'])#,color=gt1_mds['heights']['h_ph'])

In [ ]:
gt1_mds['heights'].keys()
np.unique(gt1_mds['heights']['quality_ph'])#[0, 1, 2]
np.unique(gt1_mds['heights']['signal_conf_ph'])#[-1, 0, 1, 2, 3, 4]

In [ ]:
h_avg = np.average(gt1_mds['heights']['h_ph'][:])
bds = [min(gt1_mds['heights']['lat_ph']), max(gt1_mds['heights']['lat_ph'])]

In [ ]:
plotMDS(gt1_mds, bds, h_avg)

In [ ]:
plotMDS(gt1_mds, [39.05,39.1], 2500)

In [ ]:
#https://github.com/tsutterley/read-ICESat-2/blob/main/notebooks/Read%20ICESat-2%20ATL03.ipynb
#@params: data (mds) and attributes
def seg_data(mds):
    
    #-- number of GPS seconds between the GPS epoch
    #-- and ATLAS Standard Data Product (SDP) epoch
    print(mds['ancillary_data']['atlas_sdp_gps_epoch'][0:1])
    atlas_sdp_gps_epoch = mds['ancillary_data']['atlas_sdp_gps_epoch'][:]
    
    #-- variables of interest for generating corrected elevation estimates:
    #-- ATL03 Segment ID
    Segment_ID = mds['geolocation']['segment_id']  
    n_seg = len(Segment_ID) #183
    #-- number of photon events
    n_pe, = mds['heights']['delta_time'].shape #Pond 1 -> 23033
    #-- first photon in the segment (convert to 0-based indexing)
    Segment_Index_begin = mds['geolocation']['ph_index_beg'] - 1
    #-- number of photon events in the segment:
    Segment_PE_count = mds['geolocation']['segment_ph_cnt']    

    #-- along-track distance for each ATL03 segment
    Segment_Distance = mds['geolocation']['segment_dist_x']    
    #-- along-track length for each ATL03 segment
    Segment_Length = mds['geolocation']['segment_length']

    #-- background photon rate
    #-- Transmit time of the reference photon
    delta_time = mds['geolocation']['delta_time']
    #-- interpolate background photon rate based on 50-shot summation
    background_delta_time = mds['bckgrd_atlas']['delta_time']
    SPL = scipy.interpolate.UnivariateSpline(background_delta_time,
                                             mds['bckgrd_atlas']['bckgrd_rate'],k=3,s=0)
    background_rate = SPL(delta_time)
    
    #-- along-track and across-track distance for photon events
    #atc = along/across track coordinates
    x_atc = mds['heights']['dist_ph_along'] # meters [20.090918 19.74102  20.044334 ... 16.128172 16.12856  16.129326]
    y_atc = mds['heights']['dist_ph_across']
    #-- photon event heights
    h_ph = mds['heights']['h_ph'][:]
    
    #-- for each 20m segment:
    ph_start_idx = Segment_Index_begin[0]
    for j,_ in enumerate(Segment_ID):
        #-- index for 20m segment j
        idx = Segment_Index_begin[j] - ph_start_idx #e.g. 447696
        #-- skip segments with no photon events
        if (idx < 0):
            continue
        #-- number of photons in 20m segment
        cnt = Segment_PE_count[j] #e.g. 480
        #-- add segment distance to along-track coordinates
        x_atc[idx:idx+cnt] += Segment_Distance[j] #fixed 6.30

    #-- iterate over ATLAS major frames
    photon_mframes = mds['heights']['pce_mframe_cnt']
    pce_mframe_cnt = mds['bckgrd_atlas']['pce_mframe_cnt']
    unique_major_frames,unique_index = np.unique(pce_mframe_cnt,return_index=True)
    major_frame_count = len(unique_major_frames)
    tlm_height_band1 = mds['bckgrd_atlas']['tlm_height_band1']
    tlm_height_band2 = mds['bckgrd_atlas']['tlm_height_band2']
    
    #-- photon event signal-to-noise ratio from photon classifier
    pe_weights = np.zeros((n_pe),dtype=np.float64)
    Photon_SNR = np.zeros((n_pe),dtype=int)
    #-- run for each major frame
    for iteration,idx in enumerate(unique_index):
        #-- sum of 2 telemetry band widths for major frame
        h_win_width = tlm_height_band1[idx] + tlm_height_band2[idx]
        #-- photon indices for major frame (buffered by 1 on each side)
        i1, = np.nonzero((photon_mframes >= unique_major_frames[iteration]-1) &
            (photon_mframes <= unique_major_frames[iteration]+1))
        #-- indices for the major frame within the buffered window
        i2, = np.nonzero(photon_mframes[i1] == unique_major_frames[iteration])
        #-- calculate photon event weights
        pe_weights[i1[i2]] = classify_photons(x_atc[i1], h_ph[i1],
            h_win_width, i2, K=5, MIN_PH=5, MIN_XSPREAD=1.0,
            MIN_HSPREAD=0.01, METHOD='linear')

    #-- allocate for segment means
    fill_value = np.nan
    #-- mean longitude of each segment high-confidence photons
    Segment_Lon = np.ma.zeros((n_seg),fill_value=fill_value)
    Segment_Lon.data[:] = Segment_Lon.fill_value
    Segment_Lon.mask = np.ones((n_seg),dtype=bool)
    #-- mean longitude of each segment high-confidence photons
    Segment_Lat = np.ma.zeros((n_seg),fill_value=fill_value)
    Segment_Lat.data[:] = Segment_Lat.fill_value
    Segment_Lat.mask = np.ones((n_seg),dtype=bool)
    #-- mean height of each segment high-confidence photons
    Segment_Elev = np.ma.zeros((n_seg),fill_value=fill_value)
    Segment_Elev.data[:] = Segment_Elev.fill_value
    Segment_Elev.mask = np.ones((n_seg),dtype=bool)
    #-- mean time of each segment high-confidence photons
    Segment_Time = np.ma.zeros((n_seg),fill_value=fill_value)
    Segment_Time.data[:] = Segment_Time.fill_value
    Segment_Time.mask = np.ones((n_seg),dtype=bool)
    
    #segment SNR/PE confidence (added 7.12)
    Segment_Photon_SNR = np.ma.zeros((n_seg), fill_value = fill_value)
    Segment_Photon_SNR.data[:] = Segment_Photon_SNR.fill_value
    Segment_Photon_SNR.mask = np.ones((n_seg), dtype=bool)
    
    #-- iterate over ATL03 segments to calculate 40m means
    #-- in ATL03 1-based indexing: invalid == 0
    #-- here in 0-based indexing: invalid == -1   
    segment_indices, = np.nonzero((Segment_Index_begin[:-1] >= 0) &
        (Segment_Index_begin[1:] >= 0))
    ###FIX 6.30; index references photon event numbers (first in our mds = 447696)
    ###but that's 0, for indexing purposes
    ph_start_idx = Segment_Index_begin[0]
    #print("The following segments were masked due to not enough photons events \
    #        above low-confidence threshold (10) and/or spread of photons in segment < 20m:")
    for j in segment_indices: 
        #-- index for segment j
        idx = Segment_Index_begin[j] - ph_start_idx #6.30 ***** CHANGING TO EQUAL J, SEE 20m SECTION ABOVE
        #-- number of photons in segment (use 2 ATL03 segments)
        c1 = np.copy(Segment_PE_count[j])
        c2 = np.copy(Segment_PE_count[j+1])
        cnt = c1 + c2
        #-- time of each Photon event (PE)
        segment_delta_times = mds['heights']['delta_time'][idx:idx+cnt]
        gps_seconds = atlas_sdp_gps_epoch + segment_delta_times
        time_leaps = icesat2_toolkit.time.count_leap_seconds(gps_seconds)
        #-- Photon event lat/lon and elevation (WGS84)
        segment_heights = h_ph[idx:idx+cnt].copy()
        segment_lats = mds['heights']['lat_ph'][idx:idx+cnt]
        segment_lons = mds['heights']['lon_ph'][idx:idx+cnt]
        #-- calculate segment time in Julian days (UTC)
        segment_times = 2444244.5 + (gps_seconds - time_leaps)/86400.0           
        #-- Photon event channel and identification
        ID_channel = mds['heights']['ph_id_channel'][idx:idx+cnt]
        ID_pulse = mds['heights']['ph_id_pulse'][idx:idx+cnt]
        n_pulses = np.unique(ID_pulse).__len__()
        frame_number = mds['heights']['pce_mframe_cnt'][idx:idx+cnt]
        #-- along-track X and Y coordinates
        distance_along_X = np.copy(x_atc[idx:idx+cnt])
        distance_along_Y = np.copy(y_atc[idx:idx+cnt])
        #-- check the spread of photons along-track (must be > 20m)
        along_X_spread = distance_along_X.max() - distance_along_X.min()
        #-- Along-track distance between 2 segments
        X_atc = Segment_Distance[j] + Segment_Length[j]        
        #-- check confidence level associated with each photon event for sea ice (2)
        ice_sig_conf = mds['heights']['signal_conf_ph'][idx:idx+cnt,2]
        ice_sig_low_count = np.count_nonzero(ice_sig_conf >= 1)
        #-- indices of TEP classified photons
        ice_sig_tep_pe, = np.nonzero(ice_sig_conf == -2)
        #-- photon event weights from photon classifier
        segment_weights = pe_weights[idx:idx+cnt]
        snr_norm = np.max(segment_weights)
        #-- photon event signal-to-noise ratio from photon classifier
        photon_snr = np.zeros((len(segment_weights)),dtype=int)
        
        if (snr_norm > 0):
            photon_snr[:] = 100.0*segment_weights/snr_norm
        #-- copy signal to noise ratio for photons
        Photon_SNR[idx:idx+cnt] = np.copy(photon_snr)
        #-- photon confidence levels from classifier
        pe_sig_conf = np.zeros((len(segment_weights)),dtype=int)
        #-- calculate confidence levels from photon classifier
        pe_sig_conf[photon_snr >= 25] = 2
        pe_sig_conf[photon_snr >= 60] = 3
        pe_sig_conf[photon_snr >= 80] = 4
        #-- copy classification for TEP photons
        pe_sig_conf[ice_sig_tep_pe] = -2
        pe_sig_low_count = np.count_nonzero(pe_sig_conf > 1)
        #-- check if segment has photon events classified 
        #-- for sea ice that are at least low-confidence threshold
        #-- and that the spread of photons is greater than 20m
        if (pe_sig_low_count > 10) & (along_X_spread > 20):
            #-- find photon events that are high-confidence
            ii, = np.nonzero(pe_sig_conf >= 4)
            #-- calculate mean elevation (without iterations)
            #-- NOTE that the segment elevations will NOT be corrected
            #-- for transmit pulse shape biases or first photon biases
            Segment_Elev.data[j] = icesat2_toolkit.fit.fit_geolocation(
                segment_heights[ii], distance_along_X[ii], X_atc)
            #-- calculate geolocation and time of 40m segment center
            Segment_Lon.data[j] = icesat2_toolkit.fit.fit_geolocation(
                segment_lons[ii], distance_along_X[ii], X_atc)
            Segment_Lat.data[j] = icesat2_toolkit.fit.fit_geolocation(
                segment_lats[ii], distance_along_X[ii], X_atc)
            Segment_Time.data[j] = icesat2_toolkit.fit.fit_geolocation(
                segment_times[ii], distance_along_X[ii], X_atc)
            
        #added:7/12 (average )
        Segment_Photon_SNR.data[j] = snr_norm#np.mean(segment_weights)
        Segment_Photon_SNR.mask[j] = False
            
    #-- clear photon classifier variables for beam group
    pe_weights = None
    
    #META_DICT
    seg_data = {}
    
    #-- variables of interest for generating corrected elevation estimates
    seg_data['Segment_ID'] = np.copy(Segment_ID)
    seg_data['Segment_Index_begin'] = np.copy(Segment_Index_begin)
    seg_data['Segment_PE_count'] = np.copy(Segment_PE_count)
    seg_data['Segment_Distance'] = np.copy(Segment_Distance)
    seg_data['Segment_Length'] = np.copy(Segment_Length)
    #-- mean geolocation, height and delta time
    seg_data['Segment_Lon'] = np.copy(Segment_Lon)
    seg_data['Segment_Lat'] = np.copy(Segment_Lat)
    seg_data['Segment_Elev'] = np.copy(Segment_Elev)
    seg_data['Segment_Time'] = np.copy(Segment_Time)
    #-- background photon rate
    seg_data['background_rate'] = np.copy(background_rate)
    #-- segment SNR value
    seg_data['Segment_Photon_SNR'] = np.copy(Segment_Photon_SNR)
    
    #-- photon event signal-to-noise ratio from photon classifier
    ph_SNR = np.copy(Photon_SNR)
    
    return seg_data, ph_SNR

In [ ]:
def plotMDS(mds,bds, h_avg):
            
    ht = mds['heights']['h_ph'][:]
    # dt = mds['heights']['delta_time'][:]
    lat = mds['heights']['lat_ph'][:]
    
    #-- Signal classification confidence for sea ice
    #-- 0=Land; 1=Ocean; 2=SeaIce; 3=LandIce; 4=InlandWater
    sig_conf = mds['heights']['signal_conf_ph'][:,0]
    #**** do I need to look at any of the other classification confidences?
    
        #confidence level associated with each photon event: 
        #-- -2: TEP (Transmit Echo Pulse) returns
        #-- -1: Events not associated with a specific surface type
        #--  0: noise
        #--  1: buffer but algorithm classifies as background
        #--  2: low
        #--  3: medium
        #--  4: high
    
    isTEP, = np.nonzero(sig_conf == -2)
    noSType, = np.nonzero(sig_conf == -1)
    bkgrd, = np.nonzero((sig_conf == 0) | (sig_conf == 1)) ##includes buffer
    #-- find photon events of progressively higher confidence
    lc, = np.nonzero(sig_conf == 2)
    mc, = np.nonzero(sig_conf == 3)
    hc, = np.nonzero(sig_conf == 4)
    
    #PLOT:
    fig, ax = plt.subplots(1,1,figsize=(12,4))
    plt.xlabel('Latitude (s)',fontsize=20)
    plt.ylabel('Photon Heights (m)',fontsize=20)
    ax.set_xlim(bds[0], bds[1])
    # ax.set_ylim(h_avg - 500, h_avg+500)#(-5,10)
    
    size = 2.5
    ax.plot(lat[isTEP], mds['heights']['h_ph'][isTEP],marker='.', markersize=size,lw=0,color='red',label='TEP')
    ax.plot(lat[noSType], mds['heights']['h_ph'][noSType],marker='.', markersize=size,lw=0,color='0.2',label='No Surface Classification')
    ax.plot(lat[bkgrd], mds['heights']['h_ph'][bkgrd],marker='.', markersize=size,lw=0,color='0.5',label='Background')
    ax.plot(lat[lc], mds['heights']['h_ph'][lc],marker='.', markersize=size,lw=0,color='darkorange',label='Low Confidence')
    ax.plot(lat[mc], mds['heights']['h_ph'][mc],marker='.', markersize=size,lw=0,color='mediumseagreen',label='Medium Confidence')
    ax.plot(lat[hc], mds['heights']['h_ph'][hc],marker='.', markersize=size,lw=0,color='darkorchid',label='High Confidence')
    
    #legend
    lgd = ax.legend(loc=3,frameon=False)
    lgd.get_frame().set_alpha(1.0)
    for line in lgd.get_lines():
        line.set_linewidth(6)

In [ ]:
def plotSNR(mds, SNR, bounds, pondID, h_avg):
    
    ht = mds['heights']['h_ph'][:]
    dt = mds['heights']['delta_time'][:]
    dt_min, dt_max = bounds

    #CLASSIFICATION CONFIDENCE:
    #-- Signal classification confidence for sea ice
    #-- 0=Land; 1=Ocean; 2=SeaIce; 3=LandIce; 4=InlandWater
    ice_sig_conf = mds['heights']['signal_conf_ph'][:,2]    
    #confidence level associated with each photon event: 
    isTEP, = np.nonzero(ice_sig_conf == -2)  # -2: TEP (Transmit Echo Pulse) returns
    noSType, = np.nonzero(ice_sig_conf == -1) #-- -1: Events not associated with a specific surface type
    bkgrd, = np.nonzero((ice_sig_conf == 0) | (ice_sig_conf == 1)) ##0: noise, 1: buffer (aka bckgrd)
    #-- find photon events of progressively higher confidence
    lc, = np.nonzero(ice_sig_conf == 2) #low
    mc, = np.nonzero(ice_sig_conf == 3) #medium
    hc, = np.nonzero(ice_sig_conf == 4) #high
    
    #PLOTS:
    fig,ax = plt.subplots(num=2,nrows=2,sharex=True,figsize=(12,8))
    size = 7.5 #dot size
    
    #CONFIDENCES:   
    ax[0].plot(dt[isTEP], mds['heights']['h_ph'][isTEP],marker='.', markersize=size,lw=0,color='red',label='TEP')
    ax[0].plot(dt[noSType], mds['heights']['h_ph'][noSType],marker='.', markersize=size,lw=0,color='0.2',label='No Surface Classification')
    ax[0].plot(dt[bkgrd], mds['heights']['h_ph'][bkgrd],marker='.', markersize=size,lw=0,color='0.5',label='Background')
    ax[0].plot(dt[lc], mds['heights']['h_ph'][lc],marker='.', markersize=size,lw=0,color='darkorange',label='Low Confidence')
    ax[0].plot(dt[mc], mds['heights']['h_ph'][mc],marker='.', markersize=size,lw=0,color='mediumseagreen',label='Medium Confidence')
    ax[0].plot(dt[hc], mds['heights']['h_ph'][hc],marker='.', markersize=size,lw=0,color='darkorchid',label='High Confidence')
    
    
    #SIGNAL TO NOISE RATIO:
    isort = np.argsort(SNR)
    sc = ax[1].scatter(dt[isort],mds['heights']['h_ph'][isort],
                        c=SNR[isort],s=size)

    #-- add colorbar for scatter plot
    cax = fig.add_axes([0.075, 0.080, 0.325, 0.02])
    cbar = fig.colorbar(sc, cax=cax, extend='both', extendfrac=0.0375,
    drawedges=False, orientation='horizontal')
    cbar.solids.set_rasterized(True)     # -- rasterized colorbar to remove lines
    cbar.ax.set_xlabel('Photon Classifier SNR')
    cbar.ax.xaxis.set_label_position('top')
    cbar.ax.tick_params(which='both',width=1,length=11,direction="in")
    
    #LEGEND
    lgd = ax[0].legend(loc=3,frameon=False)
    lgd.get_frame().set_alpha(1.0)
    for line in lgd.get_lines():
        line.set_linewidth(6)
    
    #AXES:
    ax[1].set_xlabel('Delta Time (s)',fontsize=10)
    ax[0].set_ylabel('Photon Heights (m)',fontsize=10)
    ax[1].set_ylabel('Photon Heights (m)',fontsize=10)
    ax[0].set_title((pondID+': Photon Event Confidence'), fontsize=15)
    ax[1].set_title((pondID+': Signal to Noise Ratio'), fontsize=15)
                    
    ax[1].set_xlim(dt_min, dt_max)
    ax[0].set_ylim(h_avg - 10,h_avg+5)
    ax[1].set_ylim(h_avg - 10,h_avg+5)
    
    #-- adjust the figure axes
    fig.subplots_adjust(left=0.07, right=0.98, bottom=0.05, top=0.95, hspace=0.05)
    
    plt.show()
